# 場景流計算

In [ ]:
# methods about scene flow
# include scene flow, depth

# import
from ast import List
import json
import pickle
import numpy as np
import cv2
import glob
import os
from pathlib import Path
from PIL import Image
# import matplotlib
import matplotlib.pyplot as plt
from sympy import Q, Array
import flow_compute
import random
from ff_core.utils import flow_viz

## 參數定義

In [ ]:
plt.rcParams['font.sans-serif'] = ['DFKai-SB']
plt.rcParams['axes.unicode_minus'] = False

MAX_SCENE_FLOW = 10

## 校正處理

In [ ]:
# ===== 校正 =====
# drop2d矩陣極端值


def drop_2d_map_bias(depth_map_2d: np.ndarray, min=-1000, max=1000):
    depth_map_2d[(depth_map_2d < min)] = min
    depth_map_2d[(depth_map_2d > max)] = max
    return depth_map_2d


# 解算P矩陣
def decompose_projection_matrix(P):
    K, R, T, _, _, _, _ = cv2.decomposeProjectionMatrix(P)
    T = T / T[3]
    return K, R, T


# 計算Q矩陣
def decompute_Q_matrix(P1, P2):
    # reference: https://stackoverflow.com/a/28317841/17732660

    f = P1[0][0]
    cx1 = P1[0][2]
    cy = P1[1][2]
    cx2 = P2[0][2]
    Tx = P2[0][3] / f

    Q = np.array([[1, 0, 0, -cx1], [0, 1, 0, -cy], [0, 0, 0, f],
                 [0, 0, -1 / Tx, (cx1 - cx2) / Tx]])
    return Q
    pass


def get_ZED_Q_matrix() -> np.ndarray:
    Q = [[1., 0., 0., -697.43959045],
         [0., 1., 0., -236.69963646],
         [0., 0., 0., 929.4452],
         [0., 0., -16.77215164, 0.]]
    return Q


# 取得KITTI Q矩陣
def get_KITTI_Q_matrix(name: str) -> np.ndarray:
    KITTI_path = Path(
        r'E:\datasets\KITTI_sceneflow\calibration_files\training\calib_cam_to_cam')
    calib_path = KITTI_path / (name + ".txt")
    calib = calib_path.read_text()
    # with calib_path.open('r') as f:
    #     calib = f.readlines()
    calib = calib.split('\n')
    # print(len(calib))
    # print(calib)

    K_02 = np.array(calib[19].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    D_02 = np.array(calib[20].strip().split()[1:]
                    ).astype(float).flatten()
    R_02 = np.array(calib[21].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    T_02 = np.array(calib[22].strip().split()[1:]
                    ).astype(float).flatten()
    S_rect_02 = np.array(calib[23].strip().split()[
                         1:]).astype(float).flatten()
    R_rect_02 = np.array(calib[24].strip().split()[
                         1:]).astype(float).reshape((3, 3))
    P_rect_02 = np.array(calib[25].strip().split()[
                         1:]).astype(float).reshape((3, 4))
    K_03 = np.array(calib[27].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    D_03 = np.array(calib[28].strip().split()[1:]
                    ).astype(float).flatten()
    R_03 = np.array(calib[29].strip().split()[1:]
                    ).astype(float).reshape((3, 3))
    R_rect_03 = np.array(calib[32].strip().split()[
                         1:]).astype(float).reshape((3, 3))
    P_rect_03 = np.array(calib[33].strip().split(
    )[1:]).astype(float).reshape((3, 4))

    R1, R2, P1, P2, Q, RECT1, RECT2 = cv2.stereoRectify(
        K_02, D_02, K_03, D_03, S_rect_02.astype(int), R_02, T_02, R_03)
    # P_rect_02_err = sum(abs(P1.flatten() - P_rect_02.flatten())) / 12
    # P_rect_03_err = sum(abs(P2.flatten() - P_rect_03.flatten())) / 12
    return Q

## 視差處理

In [ ]:
# 計算視差
def get_disparity(left_img: np.ndarray, right_img: np.ndarray):
    # 取得左右影像flow
    flow = flow_compute.prepare_mat_and_compute_flow(
        left_img, right_img)
    # 將flow結果drop掉flow的y向量
    disparityimg = np.squeeze(np.delete(np.array(flow), 1, axis=2))
    # 回傳shape=(x,y), 代表視差的矩陣
    return disparityimg


# 繪製視差
def plt_disparity_map(disparity_map: np.ndarray, title="視差圖"):
    plt.title(title)
    plt.imshow(disparity_map, cmap='cividis')
    plt.show()


# 繪製視差分布值
def plt_disparity_map_distribution(disparity_map: np.ndarray, title="視差值分布"):
    plt.title(title)
    plt.boxplot(disparity_map.flatten())
    plt.show()


# 繪製左右影像視差對應點
def plt_correspondence(left_path: str, right_path: str, disparity_map: np.ndarray, points_num=15, title="視差對應"):
    left = cv2.imread(left_path, cv2.IMREAD_COLOR)
    right = cv2.imread(right_path, cv2.IMREAD_COLOR)
    print(disparity_map.shape)
    print(left.shape)
    # 統一調整為成視差圖大小
    left = cv2.resize(left, dsize=(disparity_map.shape[1], disparity_map.shape[0]),
                      interpolation=cv2.INTER_CUBIC)
    right = cv2.resize(right, dsize=(disparity_map.shape[1], disparity_map.shape[0]),
                       interpolation=cv2.INTER_CUBIC)
    print(left.shape)
    # 隨機選取點
    rand_points = [(random.randrange(disparity_map.shape[0]), random.randrange(disparity_map.shape[1]))
                   for i in range(points_num)]
    # 準備資料
    left_points = [cv2.KeyPoint(x, y, None) for y, x in rand_points]
    right_points = [cv2.KeyPoint(x + disparity_map[y][x], y, None)
                    for y, x in rand_points]
    matches = [i for i in range(points_num)]
    matches = [cv2.DMatch(i, i, 0) for i in range(points_num)]

    # 檢查資料
    print([i.pt for i in left_points])
    print([i.pt for i in right_points])
    print(left.shape)
    print(right.shape)
    print(disparity_map.shape)

    # 使用cv2.drawMatches功能繪製匹配點
    match_result = cv2.drawMatches(
        left, left_points, right, right_points, matches, None)
    plt.title(title)
    plt.imshow(match_result)
    plt.show()
    pass

## 深度處理

In [ ]:
# 計算各pixel世界座標位置
def reproject_depth(left: np.ndarray, right: np.ndarray, Q: np.ndarray) -> np.ndarray:
    disp = get_disparity(left, right)
    _3d_map = cv2.reprojectImageTo3D(disp, Q)
    return _3d_map


# drop深度圖
def drop_depth_map(depth_map: np.ndarray):
    x = drop_2d_map_bias(depth_map[:, :, 0])
    y = drop_2d_map_bias(depth_map[:, :, 1])
    z = drop_2d_map_bias(depth_map[:, :, 2])
    return x, y, z


# 繪製深度圖
def plt_depth_map(depth_map: np.ndarray, title="深度圖"):
    plt.title(title)
    plt.imshow(depth_map[:, :, 2], cmap='cividis')
    plt.colorbar()
    plt.show()


# 繪製3D深度圖
def plt_3d_depth_map(depth_map: np.ndarray, title="深度圖3D"):
    fig = plt.figure()
    ax = plt.subplot(projection='3d')
    x = depth_map[:, :, 0].flatten()
    y = depth_map[:, :, 1].flatten()
    z = depth_map[:, :, 2].flatten()
    plt.title(title)
    ax.scatter(x, y, z, c=z)
    plt.show()


# 繪製深度圖分布
def plt_depth_map_distribution(depth_map: np.ndarray, title="深度值分布"):
    x = depth_map[:, :, 0].flatten()
    y = depth_map[:, :, 1].flatten()
    z = depth_map[:, :, 2].flatten()
    plt.title(title)
    plt.boxplot([x, y, z])
    plt.show()


def plt_depth_from_top(depth_map: np.ndarray, title="深度俯視圖"):

    pass

## 光流處理 optical flow

In [ ]:
# 計算光流
def get_flow(img0: np.ndarray, img1: np.ndarray):
    if type(img0) == type("str"):
        return flow_compute.prepare_path_and_compute_flow(img0, img1)
    return flow_compute.prepare_mat_and_compute_flow(img0, img1)


def plt_flow(flow, title="光流"):
    flow_img = flow_viz.flow_to_image(flow)
    plt.title(title)
    plt.imshow(flow_img)
    plt.colorbar()
    plt.show()

## 場景流處理 scene flow

In [ ]:
def get_scene(img0_3d, img1_3d, flow):
    # print("img0_depth_map_3d:", img0_3d.shape)
    # print("img1_depth_map_3d:", img1_3d.shape)
    # print("flow:\n", flow.shape)
    x, y, vec = img0_3d.shape
    # print("x:", x)
    # print("y:", y)
    scene = np.zeros((x, y, 3))
    for ind_x in range(x):
        for ind_y in range(y):
            target_x = int(np.round(ind_x + flow[ind_x][ind_y][0]))
            target_y = int(np.round(ind_y + flow[ind_x][ind_y][1]))
            if 0 <= target_x < x and 0 <= target_y < y:
                # print("ind_x:", ind_x)
                # print("ind_y:", ind_y)
                # print("target_x:", target_x)
                # print("target_y:", target_y)
                # print("img0_depth_map_3d[ind_x][ind_y]:",
                #       img0_3d[ind_x][ind_y])
                # print("img1_depth_map_3d[target_x][target_y]:",
                #       img1_3d[target_x][target_y])
                scene[ind_x][ind_y] = img1_3d[target_x][target_y] - \
                    img0_3d[ind_x][ind_y]
    return scene


# 繪製場景流分布
def plt_scene_flow_distribution(scene_flow, title="場景流值分布"):
    x = scene_flow[:, :, 0].flatten()
    y = scene_flow[:, :, 1].flatten()
    z = scene_flow[:, :, 2].flatten()
    plt.title(title)
    plt.boxplot([x, y, z])
    plt.show()
    pass


# 繪製場景流
def plt_scene_flow(scene_flow, title="場景流", vmin=None, vmax=None):
    """plt scene flow in different view

    Args:
        scene_flow (3d_vector): input scene flow in a matrix of 3d vector in 540*960*3 resolution
        title (str, optional): 圖表標題. Defaults to "場景流".
    """
    # build plt
    fig, axarr = plt.subplots(3)
    fig.suptitle(title)
    # x軸
    im = axarr[0].imshow(scene_flow[:, :, 0], vmin=vmin, vmax=vmax, cmap='bwr')
    plt.colorbar(im, ax=axarr[0])
    axarr[0].set_title('x 位移(正向上)')
    # y軸
    im = axarr[1].imshow(scene_flow[:, :, 1], vmin=vmin, vmax=vmax, cmap='bwr')
    plt.colorbar(im, ax=axarr[1])
    axarr[1].set_title('y 位移(正向右)')
    # z軸
    im = axarr[2].imshow(scene_flow[:, :, 2], vmin=vmin, vmax=vmax, cmap='bwr')
    plt.colorbar(im, ax=axarr[2])
    axarr[2].set_title('z 位移(正向前)')
    plt.show()
    pass


def make_colorwheel():
    """
    Generates a color wheel for optical flow visualization as presented in:
        Baker et al. "A Database and Evaluation Methodology for Optical Flow" (ICCV, 2007)
        URL: http://vision.middlebury.edu/flow/flowEval-iccv07.pdf

    Code follows the original C++ source code of Daniel Scharstein.
    Code follows the the Matlab source code of Deqing Sun.

    Returns:
        np.ndarray: Color wheel
    """

    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR
    colorwheel = np.zeros((ncols, 3))
    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.floor(255 * np.arange(0, RY) / RY)
    col = col + RY
    # YG
    colorwheel[col:col + YG, 0] = 255 - np.floor(255 * np.arange(0, YG) / YG)
    colorwheel[col:col + YG, 1] = 255
    col = col + YG
    # GC
    colorwheel[col:col + GC, 1] = 255
    colorwheel[col:col + GC, 2] = np.floor(255 * np.arange(0, GC) / GC)
    col = col + GC
    # CB
    colorwheel[col:col + CB, 1] = 255 - np.floor(255 * np.arange(CB) / CB)
    colorwheel[col:col + CB, 2] = 255
    col = col + CB
    # BM
    colorwheel[col:col + BM, 2] = 255
    colorwheel[col:col + BM, 0] = np.floor(255 * np.arange(0, BM) / BM)
    col = col + BM
    # MR
    colorwheel[col:col + MR, 2] = 255 - np.floor(255 * np.arange(MR) / MR)
    colorwheel[col:col + MR, 0] = 255
    return colorwheel

def get_color(u, v, vol):

    return r, g, b


# 把場景流視覺化。分為xy平面(後視圖)、yz平面(側視圖)、zx平面(上視圖)
def plt_visualize_scene_flow(scene_flow, title="場景流視覺化"):

    x_flow = scene_flow[:, :, 0]
    y_flow = scene_flow[:, :, 1]
    z_flow = scene_flow[:, :, 2]
    plt.title("xy平面(後視圖)")
    xy_flow = np.dstack((x_flow, y_flow))
    print(xy_flow)

    plt.title("yz平面(側視圖)")
    yz_flow = np.dstack((y_flow, z_flow))
    print(yz_flow)
    plt.title("zx平面(上視圖)")
    zx_flow = np.dstack((z_flow, x_flow))
    print(zx_flow)
    pass